In [24]:
import requests
import re
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### Резюме

Ищем ссылки на резюме с ключевым запросом IT, геолокацией -- Россия (по умолчанию отдается Москва), с поиском ключевого запроса (фильтры) 'по всем словам', 'в тексте резюме', пока не начали постоянно отдаваться дубликаты, ограничения на номер страниц в резюме superjob нет

In [56]:
main_url = "https://www.superjob.ru"
search_url = main_url + "/resume/search_resume.html"

resume_urls = np.array([])
all_dublicates_count = 0
i = 1
while all_dublicates_count < 30:
    url_params = {
        'c[0]': '1', # это геолокация == Россия
        'keywords[0][keys]': 'IT',
        'keywords[0][skwc]': 'and',
        'keywords[0][srws]': '7',
        'page': i
    }
    resume_html = requests.get(search_url, params=url_params).text
    soup = BeautifulSoup(resume_html, 'html.parser')
    resume_div = soup.find('div', class_='_1Ttd8 sESpW')
    resume_div = resume_div.find_all('div', class_='_3VcZr')[1]
    resume_divs = resume_div.find_all('div', class_='_1ruK8')
    resume_links = resume_div.find_all('a', class_='icMQ_ f-test-link- _3dPok')
    urls = list(map(lambda link: main_url + link.attrs['href'], resume_links))
    urls = np.array(urls)
    diff = np.unique(np.append(resume_urls, urls)).shape[0] - resume_urls.shape[0] 
    if diff == 0:
        all_dublicates_count += 1
    resume_urls = np.unique(np.append(resume_urls, urls))
    if (i % 10) == 0:
        print("page:", i, "collected:", resume_urls.shape[0])
    i += 1
print("page_count:", i)

page: 10 collected: 300
page: 20 collected: 600
page: 30 collected: 900
page: 40 collected: 1200
page: 50 collected: 1500
page: 60 collected: 1800
page: 70 collected: 2100
page: 80 collected: 2400
page: 90 collected: 2700
page: 100 collected: 3000
page: 110 collected: 3300
page: 120 collected: 3600
page: 130 collected: 3900
page: 140 collected: 4200
page: 150 collected: 4500
page: 160 collected: 4800
page: 170 collected: 5000
page: 180 collected: 5000
page: 190 collected: 5000
page_count: 198


In [54]:
print(len(resume_urls))
resume_urls[:10]

5000


array(['https://www.superjob.ru/resume/3d-modeller-43623349.html',
       'https://www.superjob.ru/resume/administrator-26936498.html',
       'https://www.superjob.ru/resume/administrator-2832565.html',
       'https://www.superjob.ru/resume/administrator-35752711.html',
       'https://www.superjob.ru/resume/administrator-37440177.html',
       'https://www.superjob.ru/resume/administrator-42151588.html',
       'https://www.superjob.ru/resume/administrator-43041335.html',
       'https://www.superjob.ru/resume/administrator-43894667.html',
       'https://www.superjob.ru/resume/administrator-baz-dannyh-9452765.html',
       'https://www.superjob.ru/resume/administrator-it-proekta-44001872.html'],
      dtype='<U117')

Соберу те резюме, у которых указана зарплата

In [353]:
def get_salary(soup):
    if not soup:
        return
    header = list(soup.find('div', class_='_1_bQo _2FJA4').children)
    header2 = list(header[1].find('span', class_='_1h3Zg _1fqdH ADNB4').children)[0]
    header2 = list(header2)
    salary = list(header2[0].find('span', class_='_1h3Zg ADNB4 _1BoTZ').children)[0]
    salary = list(salary.children)
    if salary[0] == 'По договорённости':
        return
    return {'salary': salary[0].text + salary[2].text}

Категории, в которых лежит резюме

In [340]:
def get_categories(soup):
    category = list(soup.children)[0]
    category = soup.find_all('span', class_='_1h3Zg z15vz _1xK5K')[1]
    category = [c.text for c in category.find_all('a')]
    category.remove('Работа, резюме и вакансии')
    category.remove('Резюме')
    return {'categories': ', '.join(category)}

Информация про возраст, пол, город, фото, гражданство и прочее

In [341]:
def get_bio_info(soup):
    result = {}
    main_info = list(soup.children)[1]
    photo = main_info.find('div', class_='jGStW').find('img')
    if photo:
        result['photo_url'] = 'https:' + photo.attrs['src']
    main_info = main_info.find('div', class_='_2XXYS _2cxK3')
    main_info = main_info.find_all('div', class_='_2g1F-')
    age_gender = list(main_info[0].find('span').children)
    age = age_gender[0].find('span').find('span').text
    age = re.sub('[\sа-я]', '', age)
    result['age'] = age
    gender = age_gender[1].text
    if 'женщина' in gender:
        result['gender'] = 'f'
    elif 'мужчина' in gender:
        result['gender'] = 'm'
    if len(age_gender) > 2:
        result['family'] = re.sub('^, ', '', age_gender[2].text)
        if len(age_gender) > 3:
            result['family'] += age_gender[3].text
    city_moving = list(main_info[1].find('span').children)
    if len(city_moving) > 0:
        result['city'] = city_moving[0].text
        if len(city_moving) > 1:
            result['metro_or_moving'] = re.sub(', ', '', city_moving[1].text)
    cit = main_info[2].find('span')
    if cit:
        cit = cit.find('span').text
        result['citizenship'] = re.sub('Гражданство: ', '', cit)
    return result

Информация об опыте работы, образовании и навыках

In [348]:
def get_experience(soup):
    result = []
    ul = soup.find('ul', class_='_36e54 _2ntFx _1qf3L')
    for li in list(ul.children):
        exp = {}
        div = li.find('div', class_='_9tygw')
        periods = list(div.find('ul', class_='_1c9Iy _2XXYS _2cxK3 _1qf3L').children)
        exp['period'] = periods[0].find('span').text
        exp['length'] = periods[1].find('span', class_='_3AnJT').find('span').text
        work_info = list(div.find('div', class_='_3ipSP').children)
        title = work_info[0]
        exp['name'] = title.find('h3', class_='_1h3Zg _1hd4a ADNB4 _1BoTZ').text
        company = title.find('div', class_='_1hd4a')
        if company:
            exp['company'] = company.find('span').text
        duties_ach = list(work_info[1].children)
        if len(duties_ach) > 0:
            exp['duties'] = duties_ach[0].find('div', class_='_2VtGa _1hd4a').text
            exp['duties'] = re.sub('\n', ' ', exp['duties'])
            if len(duties_ach) > 1:
                exp['achievments'] = duties_ach[1].find('div', class_='_2VtGa _1hd4a').text
                exp['achievments'] = re.sub('\n', ' ', exp['achievments'])
        result.append(exp)
    return result

def get_education(soup):
    result = []
    ul = soup.find('ul', class_='_36e54 _2ntFx _1qf3L')
    for li in list(ul.children):
        ed = {}
        div = li.find('div', class_='_9tygw')
        periods = list(div.find('ul', class_='_1c9Iy _2XXYS _2cxK3 _1qf3L').children)
        ed['kind'] = periods[0].find('div').text
        if len(periods) > 1:
            text = periods[1].find('div').text
            if re.match('^\d+$', text):
                ed['period'] = text
            else:
                ed['form'] = text
            if len(periods) > 2:
                ed['period'] = periods[2].find('div').text
        info = div.find('div', class_='_7IHu7')
        univ = info.find('h3')
        if univ:
            ed['university'] = univ.text
        un_info = info.find('div', class_='NIzpP _2XXYS _2cxK3')
        if un_info:
            un_info = list(un_info.children)
            for ui in un_info:
                text = ui.find('div', class_='_1h3Zg _1hd4a z15vz ADNB4 _21a7u').text
                if 'Факультет: ' in text:
                    ed['faculty'] = re.sub('Факультет: ', '', text)
                elif 'Специальность: ' in text:
                    ed['specialty'] = re.sub('Специальность: ', '', text)
        result.append(ed)
    return result

def get_languages(soup):
    result = []
    ul = soup.find('ul', class_='_1_bQo _2FJA4 _1qf3L')
    for li in list(ul.children):
        result.append(li.find('div').text)
    return result

def get_skills(soup):
    result = []
    skills = list(soup.find('div', class_='uI8sE').children)
    for skill in skills:
        s = {}
        name = skill.find('h4').text
        if 'Профессиональные навыки:' in name:
            s['professional'] = skill.find('div', class_='_2VtGa _1hd4a').text
            s['professional'] = re.sub('\n', ' ', s['professional'])
        elif 'Дополнительные сведения:' in name:
            s['extra'] = skill.find('div', class_='_2VtGa _1hd4a').text
            s['extra'] = re.sub('\n', ' ', s['extra'])
        result.append(s)
    return result

def get_auto(soup):
    result = []
    ul = soup.find('ul', class_='_36e54 _2ntFx _1qf3L')
    for li in list(ul.children):
        result.append(li.find('div').text)
    return result

def get_work_body_info(soup):
    result = {}
    body = list(soup.children)
    for block in body:
        name = block.find('div', class_='riZcC').find('h2').text
        if 'Опыт работы' in name:
            result['experience_length'] = re.sub('Опыт работы ', '', name)
            result['experience'] = get_experience(block)
        elif 'Образование' in name:
            result['education'] = get_education(block)
        elif 'Иностранные языки' in name:
            result['languages'] = get_languages(block)
        elif 'Знания и навыки' in name:
            result['skills'] = get_skills(block)
        elif 'Водительские права' in name:
            result['auto'] = get_auto(block)
    return result

def get_work_header_info(soup):
    result = {}
    header = list(soup.find('div', class_='_1_bQo _2FJA4').children)
    result['name'] = header[0].find('h1').text
    header2 = list(header[1].find('span', class_='_1h3Zg _1fqdH ADNB4').children)[0]
    header2 = list(header2)
    if len(header2) > 1:
        empl = header2[1].find('span')
        if empl:
            result['employment'] = empl.text
        else:
            result['business_trips'] = re.sub('^, ', '', header2[1].text)
        if len(header2) > 2:
            result['business_trips'] = re.sub('^, ', '', header2[2].text)
    return result

def get_work_info(soup):
    result = {}
    info = list(soup.children)[2]
    info = list(info.find('div', class_='_1Ttd8 _2CsQi').children)
    if len(info) < 3:
        return # нам не нужны резюме вида https://www.superjob.ru/resume/direktor-po-it-43971680.html
    result.update(get_work_header_info(info[0]))
    result.update(get_work_body_info(info[2]))
    return result

Ну и сбор всего вместе

In [358]:
def collect_resume(soup):
    result = {}
    salary = get_salary(soup)
    if not salary:
        return
    result.update(salary)
    result.update(get_categories(soup))
    result.update(get_bio_info(soup))
    work_info = get_work_info(soup)
    if not work_info:
        return
    result.update(work_info)
    return result

result = []
for url_id, url in enumerate(resume_urls):
    resume_html = requests.get(url).text
    resume_html = resume_html.replace('\xa0',' ')
    soup = BeautifulSoup(resume_html, 'html.parser')
    soup = soup.find('div', class_='_1x409')
    soup = soup.find('div', class_='_36e54 _2ntFx')
    try:
        resume = collect_resume(soup)
        if resume:
            resume['url'] = url
            result.append(resume)
    except:
        print(url_id, url)
    if (url_id % 10) == 0:
         print("url", url_id, "processed, collected", len(result), 'resume with salary...', end=' ')

#check some url
# url = resume_urls[2043]
# print(url)
# resume_html = requests.get(url).text
# resume_html = resume_html.replace('\xa0',' ')
# soup = BeautifulSoup(resume_html, 'html.parser')
# soup = soup.find('div', class_='_1x409')
# soup = soup.find('div', class_='_36e54 _2ntFx')
# resume = collect_resume(soup)
# if resume:
#     resume['url'] = url
#     for k, v in resume.items():
#         print(''*10)
#         print(k, ":", v)
#     result.append(resume)
# else:
#     print('no salary datermined')

url 0 processed, collected 1 resume with salary... url 10 processed, collected 7 resume with salary... url 20 processed, collected 11 resume with salary... url 30 processed, collected 15 resume with salary... url 40 processed, collected 17 resume with salary... url 50 processed, collected 20 resume with salary... url 60 processed, collected 22 resume with salary... url 70 processed, collected 24 resume with salary... url 80 processed, collected 25 resume with salary... url 90 processed, collected 28 resume with salary... url 100 processed, collected 33 resume with salary... url 110 processed, collected 39 resume with salary... url 120 processed, collected 43 resume with salary... url 130 processed, collected 46 resume with salary... url 140 processed, collected 48 resume with salary... url 150 processed, collected 52 resume with salary... url 160 processed, collected 55 resume with salary... url 170 processed, collected 58 resume with salary... url 180 processed, collected 62 resume wi

url 1490 processed, collected 669 resume with salary... url 1500 processed, collected 674 resume with salary... url 1510 processed, collected 677 resume with salary... url 1520 processed, collected 680 resume with salary... url 1530 processed, collected 686 resume with salary... url 1540 processed, collected 693 resume with salary... url 1550 processed, collected 697 resume with salary... url 1560 processed, collected 700 resume with salary... url 1570 processed, collected 700 resume with salary... url 1580 processed, collected 702 resume with salary... url 1590 processed, collected 708 resume with salary... url 1600 processed, collected 712 resume with salary... url 1610 processed, collected 717 resume with salary... url 1620 processed, collected 722 resume with salary... url 1630 processed, collected 727 resume with salary... url 1640 processed, collected 731 resume with salary... url 1650 processed, collected 736 resume with salary... url 1660 processed, collected 739 resume with sa

url 2940 processed, collected 1352 resume with salary... url 2950 processed, collected 1356 resume with salary... url 2960 processed, collected 1359 resume with salary... url 2970 processed, collected 1363 resume with salary... url 2980 processed, collected 1370 resume with salary... url 2990 processed, collected 1376 resume with salary... url 3000 processed, collected 1383 resume with salary... url 3010 processed, collected 1388 resume with salary... url 3020 processed, collected 1393 resume with salary... url 3030 processed, collected 1398 resume with salary... url 3040 processed, collected 1403 resume with salary... url 3050 processed, collected 1406 resume with salary... url 3060 processed, collected 1412 resume with salary... url 3070 processed, collected 1418 resume with salary... url 3080 processed, collected 1421 resume with salary... url 3090 processed, collected 1426 resume with salary... url 3100 processed, collected 1432 resume with salary... url 3110 processed, collected 1

url 4380 processed, collected 2058 resume with salary... url 4390 processed, collected 2064 resume with salary... url 4400 processed, collected 2070 resume with salary... url 4410 processed, collected 2073 resume with salary... url 4420 processed, collected 2075 resume with salary... url 4430 processed, collected 2081 resume with salary... url 4440 processed, collected 2086 resume with salary... url 4450 processed, collected 2091 resume with salary... url 4460 processed, collected 2096 resume with salary... url 4470 processed, collected 2100 resume with salary... url 4480 processed, collected 2102 resume with salary... url 4490 processed, collected 2107 resume with salary... url 4500 processed, collected 2111 resume with salary... url 4510 processed, collected 2118 resume with salary... url 4520 processed, collected 2126 resume with salary... url 4530 processed, collected 2131 resume with salary... url 4540 processed, collected 2137 resume with salary... url 4550 processed, collected 2

Осталось резюме с указанной зарплатой:

In [360]:
len(result)

2369

In [359]:
# и сохранение
df = pd.DataFrame(result)
print(df.shape)
df.head()

(2369, 19)


,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"[Немецкий язык, Базовый]",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...
1,39,[B — легковые авто],NaN,Техническая поддержка,Россия,Москва,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'октябрь 2017 – работает сейчас', ...",18 лет,cостоит в браке,m,NaN,NaN,"Администратор, управляющий, IT специалист",https://public.superjob.ru/images/resume_fotos...,50 000₽,[{'professional': '• 1С: Предприятие • 1С: Роз...,https://www.superjob.ru/resume/administrator-2...
2,47,NaN,NaN,Передача данных и доступ в интернет,NaN,Новосибирск,"[{'kind': 'Высшее образование', 'form': 'Вечер...",полная занятость,"[{'period': 'февраль 2008 – работает сейчас', ...",12 лет и 10 месяцев,"cостоит в браке, есть дети",m,"[Английский язык, Технический]",Берёзовая роща,Администратор / Инженер технической поддержки IT,NaN,30 000₽,[{'professional': 'Английский язык уровень. Pr...,https://www.superjob.ru/resume/administrator-2...
3,23,NaN,NaN,Техническая поддержка,NaN,Самара,"[{'kind': 'Высшее образование', 'form': 'Дневн...",сменный график работы,"[{'period': 'июль 2015 – работает сейчас', 'le...",4 года и 3 месяца,NaN,f,NaN,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",https://public.superjob.ru/images/resume_fotos...,25 000₽,"[{'professional': 'Веб-дизайн, photoshop, MS O...",https://www.superjob.ru/resume/administrator-3...
4,52,NaN,не готов к командировкам,АХО,Россия,Ангарск,"[{'kind': 'Высшее образование', 'form': 'Дневн...",NaN,"[{'period': 'июнь 1993 – март 2015', 'length':...",21 год и 10 месяцев,"не состоит в браке, есть дети",f,NaN,NaN,"Администратор, IT-специалист, диспетчер",NaN,20 000₽,NaN,https://www.superjob.ru/resume/administrator-3...


In [361]:
with open('data/resume.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv)